# 處置股事件研究

In [32]:
# [Env Setup] 載入必要套件與設定
import pandas as pd
from tqdm import tqdm
from loguru import logger
import sys
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
sys.path.append("/Users/xinc./Documents/GitHub/note")
sys.path.append(os.getcwd()) # 加入目前路徑以匯入 utils

from module.get_info_FinMind import FinMindClient, FinMindConfig
from module.get_info_Finlab import FinlabClient
from module.plot_func import plot
from utils import batch_fetch_prices, run_event_study, process_disposal_events

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Part 1: Data Preparation

### 核心功能
1. **資料整合**：自動對接 Finlab (處置資訊) 與 FinMind (股價) API。
2. **智慧分級**：實作連續處置判斷邏輯 (Strict Overlap)，自動識別第 1, 2...N 次處置事件。
3. **動態標籤**：產生時間軸標籤 `s+N` (處置開始/期間) 與 `e+N` (處置結束後)，包含解禁日 `e+0`。
4. **雙軌輸出**：
   - **Wide Format (`disposal_df_wide.csv`)**：訊號表 (Signal Table)，不含價格，專供回測系統產生交易訊號。
   - **Long Format (`disposal_df_long.csv`)**：分析表 (Analysis Table)，含完整價量與報酬率，專供統計研究與視覺化。

### 使用流程
- **Step 1**: 抓取處置公告。
- **Step 1.5**: 執行前處理 (分級與濾網)。
- **Step 2**: 平行化抓取處置期間股價。
- **Step 3**: 執行 Event Study 轉換，產出 Wide/Long CSV。

In [33]:
# [Step 1] 抓取 Finlab 處置股資料
# 若需要 Token，請在初始化時傳入，例如 FinlabClient(token="YOUR_TOKEN")
finlab_client = FinlabClient()
print("Fetching disposal information from Finlab...")

# 抓取資料 (設定較大的範圍以確保涵蓋需求)
finlab_disposal = finlab_client.get_data("disposal_information", start_date='2018-01-01')

# [Manual Filter] 手動篩選日期 (修正 Finlab API 時間過濾限制)
if not finlab_disposal.empty:
    finlab_disposal['date'] = pd.to_datetime(finlab_disposal['date'])
    finlab_disposal = finlab_disposal[finlab_disposal['date'] >= '2018-01-01']
    print(f"Fetched {len(finlab_disposal)} records from Finlab.")
    print(f"Data Range: {finlab_disposal['date'].min()} to {finlab_disposal['date'].max()}")
else:
    print("No data fetched from Finlab.")

Fetching disposal information from Finlab...
Fetched 3383 records from Finlab.
Data Range: 2018-01-04 00:00:00 to 2025-09-26 00:00:00


In [38]:
# [Step 1.5] 前處理與分級 (Preprocessing)
# 這一步會先將 Finlab 處置資料整理格式，並標記 First/Second Disposal
if 'finlab_disposal' in locals() and not finlab_disposal.empty:
    print("Processing disposal events...")
    processed_disposal = process_disposal_events(finlab_disposal)
    
    # print(f"Processed Data Shape: {processed_disposal.shape}")
    # display(processed_disposal.head())
    
    # Optional: Save for inspection
    processed_disposal.to_csv('../../data/disposal/processed_disposal_events.csv', index=False, encoding='utf-8-sig')
else:
    print("Finlab data not available. Please run Step 1 first.")
    processed_disposal = pd.DataFrame()

Processing disposal events...
Columns before processing: ['Stock_id', 'date', '證券名稱', 'condition', '處置措施', '處置內容', 'event_start_date', 'event_end_date', 'interval', 'key_date']
Processed 3383 events.
Level Distribution:
disposal_level
1     2702
2      453
3      110
4       36
5       17
6        8
7        6
8        5
9        5
10       4
11       4
12       4
13       3
14       3
15       3
16       3
17       3
18       3
19       3
20       2
21       2
22       2
23       1
24       1
Name: count, dtype: int64


In [35]:
# [Step 2] 平行化抓取股價 (FinMind)
# 使用處理過的事件表 (processed_disposal) 以確保連續處置區間不遺漏
logger.remove()
logger.add(sys.stderr, level="WARNING")

# Initialize FinMind Client
fm_client = FinMindClient()

offset_days = 5

if 'processed_disposal' in locals() and not processed_disposal.empty:
    # 開始抓取
    price_df = batch_fetch_prices(fm_client, processed_disposal, offset_days=offset_days, max_workers=10)

    if not price_df.empty:
        print(f"Fetched Price Data Shape: {price_df.shape}")
        display(price_df.head())
    else:
        print("No price data fetched.")
else:
    print("No processed disposal data found. Please run Step 1.5.")

gc.collect()

Using pre-processed columns 'event_start_date' and 'event_end_date'.
Starting batch fetch for 1317 stocks with 10 workers...


Fetching Prices: 100%|██████████| 1317/1317 [00:39<00:00, 33.57it/s]

Fetched total 37289 rows.
Fetched Price Data Shape: (37289, 8)


,Date,Stock_id,Open,High,Low,Close,Volume,TradingAmount
0,2021-07-02,030001,24.5,26.8,22.1,26.6,16000,409800
1,2021-07-05,030001,26.6,26.6,25.5,26.0,4000,104100
2,2021-07-06,030001,27.0,28.0,27.0,27.0,8000,218100
3,2021-07-07,030001,27.0,27.0,25.3,25.3,16000,421800
4,2021-07-08,030001,25.3,25.3,25.0,25.3,8000,201200


602

In [ ]:
# [Step 3] 執行 Event Study 分析
# 使用 processed_disposal，其中已經包含 is_first_disposal 等標記
disposal_wide, disposal_long = run_event_study(price_df, processed_disposal, offset_days=offset_days)

if not disposal_wide.empty:
    print(f"Wide Format Shape: {disposal_wide.shape}")
    print(f"Long Format Shape: {disposal_long.shape}")
    
    print("\n[Wide Head]")
    display(disposal_wide.head())
    
    print("\n[Long Head]")
    display(disposal_long.head())
    
    # Save both files
    disposal_wide.to_csv('../../data/disposal/disposal_df_wide.csv', index=False, encoding='utf-8-sig')
    disposal_long.to_csv('../../data/disposal/disposal_df_long.csv', index=False, encoding='utf-8-sig')
    print("Saved 'disposal_df_wide.csv' and 'disposal_df_long.csv'.")

else:
    print("Analysis returned empty DataFrame.")

Detected Disposal Levels: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23)]
Converting to Wide Format...
Analysis completed. Wide shape: (36797, 186), Long shape: (43844, 48)
Wide Format Shape: (36797, 186)
Long Format Shape: (43844, 48)

[Wide Head]


,Date,Stock_id,t_label_first,condition_first,interval_first,event_start_date_first,event_end_date_first,relative_day_first,gap_days_first,calendar_relative_day_first,...,gap_days_level_22,calendar_relative_day_level_22,t_label_level_23,condition_level_23,interval_level_23,event_start_date_level_23,event_end_date_level_23,relative_day_level_23,gap_days_level_23,calendar_relative_day_level_23
0,2020-03-23,00642U,s-4,監視業務督導會報決議,5.0,2020-03-27,2020-04-13,-4.0,0.0,-4.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,2020-03-24,00642U,s-3,監視業務督導會報決議,5.0,2020-03-27,2020-04-13,-3.0,0.0,-3.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,2020-03-25,00642U,s-2,監視業務督導會報決議,5.0,2020-03-27,2020-04-13,-2.0,0.0,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,2020-03-26,00642U,s-1,監視業務督導會報決議,5.0,2020-03-27,2020-04-13,-1.0,0.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,2020-03-27,00642U,s+0,監視業務督導會報決議,5.0,2020-03-27,2020-04-13,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN



[Long Head]


,Date,Stock_id,Open,High,Low,Close,Volume,TradingAmount,trading_idx,prev_trade_date,...,t_label_level_15,t_label_level_16,t_label_level_17,t_label_level_18,t_label_level_19,t_label_level_20,t_label_level_21,t_label_level_22,t_label_level_23,daily_ret
0,2020-03-23,00642U,9.30,10.10,9.20,10.10,56754750,552693946,0,NaT,...,None,None,None,None,None,None,None,None,None,0.086022
12,2020-03-24,00642U,10.32,10.57,10.17,10.36,42796200,444039684,1,2020-03-23,...,None,None,None,None,None,None,None,None,None,0.003876
24,2020-03-25,00642U,10.55,10.59,10.44,10.46,39627340,416239309,2,2020-03-24,...,None,None,None,None,None,None,None,None,None,-0.008531
36,2020-03-26,00642U,10.44,10.46,10.11,10.33,25842235,264724135,3,2020-03-25,...,None,None,None,None,None,None,None,None,None,-0.010536
48,2020-03-27,00642U,10.24,10.24,10.10,10.17,20479766,207667697,4,2020-03-26,...,None,None,None,None,None,None,None,None,None,-0.006836


Saved 'disposal_df_wide.csv' and 'disposal_df_long.csv'.


In [ ]:
# [Step 4] 最終篩選 (Final Filter: Common Stocks Only)
# 應用篩選邏輯：只保留代碼長度為 4 且非 00 開頭的股票 (只保留股票)

def is_common_stock(stock_id):
    sid = str(stock_id)
    return len(sid) == 4 and not sid.startswith('00')

if 'disposal_wide' in locals() and not disposal_wide.empty:
    print("Filtering Final Output for Common Stocks Only...")
    
    # Filter Wide Format
    mask_wide = disposal_wide['Stock_id'].apply(is_common_stock)
    final_wide = disposal_wide[mask_wide].copy()
    
    # Filter Long Format
    mask_long = disposal_long['Stock_id'].apply(is_common_stock)
    final_long = disposal_long[mask_long].copy()
    
    print(f"Wide Format: {len(disposal_wide)} -> {len(final_wide)} rows")
    print(f"Long Format: {len(disposal_long)} -> {len(final_long)} rows")
    
    # Save filtered versions
    final_wide.to_csv('../../data/disposal/disposal_df_wide.csv', index=False, encoding='utf-8-sig')
    final_long.to_csv('../../data/disposal/disposal_df_long.csv', index=False, encoding='utf-8-sig')
    print("Saved 'disposal_df_wide.csv' and 'disposal_df_long.csv'.")
    
    # Preview
    display(final_wide.head())
else:
    print("Output dataframes not found. Please run Step 3 first.")

Filtering Final Output for Common Stocks Only...
Wide Format: 36797 -> 34765 rows
Long Format: 43844 -> 40532 rows
Saved 'disposal_df_wide.csv' and 'disposal_df_long.csv'.


,Date,Stock_id,t_label_first,condition_first,interval_first,event_start_date_first,event_end_date_first,relative_day_first,gap_days_first,calendar_relative_day_first,...,gap_days_level_22,calendar_relative_day_level_22,t_label_level_23,condition_level_23,interval_level_23,event_start_date_level_23,event_end_date_level_23,relative_day_level_23,gap_days_level_23,calendar_relative_day_level_23
890,2020-07-16,1213,s-3,連續三次,NaN,2020-07-21,2020-08-03,-3.0,0.0,-5.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
891,2020-07-17,1213,s-2,連續三次,NaN,2020-07-21,2020-08-03,-2.0,0.0,-4.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
892,2020-07-20,1213,s-1,連續三次,NaN,2020-07-21,2020-08-03,-1.0,2.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
893,2020-07-21,1213,s+0,連續三次,NaN,2020-07-21,2020-08-03,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
894,2020-07-22,1213,s+1,連續三次,NaN,2020-07-21,2020-08-03,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


---
# Part 2: 處置效應分析 (Statistical Analysis & Visualization)

在此區塊中，我們將使用先前產出的 **Long Format (`disposal_df_long`)** 進行深入的統計分析。
目標包含：
1. **平均報酬率 (Average Return)**：各級別處置在不同時間點 (t-label) 的表現。
2. **勝率分析 (Win Rate)**：上漲機率。
3. **分群比較**：比較不同處置條件、時間長短的效應差異。

請確保您已完成上方的 Step 1~4 並成功產生了資料。

### 📊 資料欄位說明 (Data Dictionary)

在開始分析前，了解資料集中的關鍵變數定義：

| 欄位名稱 | 說明 | 用途 |
| :--- | :--- | :--- |
| **`t_label`** | 時間軸標籤 | **核心分析欄位**。`s+N` (處置期間), `e+N` (解禁後)。`e+0` 為解禁當日。 |
| **`trading_idx`** | 交易日序號 | 該股票的累積交易日數 (Index)。用來計算精確的 `relative_day`，避免假日干擾。 |
| **`gap_days`** | 交易間隔(日曆日) | 距離上一個交易日過了幾天。`1`=正常, `3`=跨週末, `>3`=長假或暫停交易。 |
| **`relative_day`** | 相對天數 | 距離事件開始日 (`s+0`) 的交易日數差。 |
| **`disposal_level`** | 處置連續等級 | `1`=首次, `2`=續處置 (中間無間斷)。用於觀察連續處置的邊際效應遞減。 |
| **`daily_ret`** | 當日報酬率 | 計算方式：`(Close / Open) - 1`。 |
| **`Stock_id`** | 股票代號 | 經過 Filter 後應僅包含普通股 (4碼)。 |

In [39]:
long_path = '../../data/disposal/disposal_df_long.csv'
df = pd.read_csv(long_path)
df

/var/folders/np/82kc955j7g9_6x0dw88zjzw40000gn/T/ipykernel_1342/770865948.py:2: DtypeWarning:

Columns (29,30,31,32,33,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.



,Date,Stock_id,Open,High,Low,Close,Volume,TradingAmount,trading_idx,prev_trade_date,...,t_label_level_15,t_label_level_16,t_label_level_17,t_label_level_18,t_label_level_19,t_label_level_20,t_label_level_21,t_label_level_22,t_label_level_23,daily_ret
0,2020-07-16,1213,8.92,8.92,8.92,8.92,41813,372971,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,2020-07-17,1213,9.81,9.81,9.05,9.81,1009465,9853081,1,2020-07-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2,2020-07-20,1213,10.75,10.75,9.43,10.00,503610,5266828,2,2020-07-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.069767
3,2020-07-20,1213,10.75,10.75,9.43,10.00,503610,5266828,2,2020-07-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.069767
4,2020-07-21,1213,11.00,11.00,10.25,10.35,182100,1951863,3,2020-07-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.059091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40527,2021-08-05,9962,24.00,24.00,22.60,22.80,1441000,33237700,26,2021-08-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.050000
40528,2021-08-06,9962,22.10,23.05,20.55,21.15,2679000,57994100,27,2021-08-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.042986
40529,2021-08-09,9962,19.85,22.95,19.20,21.10,13423000,283985300,28,2021-08-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.062972
40530,2021-08-10,9962,21.10,21.40,19.30,19.45,6969000,139699850,29,2021-08-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.078199
